In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,35906
2,Huelva,Confirmados PDIA 14 días,1885
3,Huelva,Tasa PDIA 14 días,"367,3246682385954"
4,Huelva,Confirmados PDIA 7 días,1095
5,Huelva,Tasa PDIA 7 dias,"213,3795818149931"
6,Huelva,Total Confirmados,36101
7,Huelva,Curados,32899
8,Huelva,Fallecidos,395


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  35906.0
/tmp/ipykernel_364249/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  9980.0
/tmp/ipykernel_364249/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_364249/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_364249/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_364249/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 342 personas en los últimos 7 días 

Un positivo PCR cada 211 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,35906.0,1095.0,1885.0,513170.0,213.379582,367.324668,144.0
Huelva-Costa,20991.0,772.0,1330.0,289548.0,266.622460,459.336621,94.0
Huelva (capital),9980.0,421.0,683.0,143837.0,292.692423,474.843052,51.0
Condado-Campiña,11453.0,240.0,425.0,156231.0,153.618680,272.033079,43.0
Aljaraque,1300.0,66.0,103.0,21474.0,307.348421,479.649809,12.0
Lepe,2354.0,65.0,134.0,27880.0,233.142037,480.631277,12.0
Ayamonte,1501.0,62.0,104.0,21104.0,293.783169,492.797574,10.0
Gibraleón,707.0,18.0,32.0,12737.0,141.320562,251.236555,8.0
Sierra de Huelva-Andévalo Central,3090.0,82.0,116.0,67391.0,121.677969,172.129810,8.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Niebla,310.0,11.0,44.0,4116.0,267.249757,1068.999028,1.0
Isla Cristina,2446.0,67.0,165.0,21393.0,313.186556,771.280325,5.0
Cortelazor,15.0,2.0,2.0,299.0,668.896321,668.896321,0.0
Palos de la Frontera,808.0,43.0,68.0,11742.0,366.206779,579.117697,5.0
Santa Bárbara de Casa,37.0,2.0,6.0,1043.0,191.754554,575.263663,0.0
Ayamonte,1501.0,62.0,104.0,21104.0,293.783169,492.797574,10.0
Lepe,2354.0,65.0,134.0,27880.0,233.142037,480.631277,12.0
Aljaraque,1300.0,66.0,103.0,21474.0,307.348421,479.649809,12.0
Zarza-Perrunal (La),36.0,3.0,6.0,1253.0,239.425379,478.850758,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Minas de Riotinto,134.0,1.0,4.0,3812.0,26.232949,104.931794,1.0,0.250000
Niebla,310.0,11.0,44.0,4116.0,267.249757,1068.999028,1.0,0.250000
Alosno,209.0,2.0,7.0,3933.0,50.851767,177.981185,0.0,0.285714
Aracena,562.0,5.0,17.0,8255.0,60.569352,205.935796,1.0,0.294118
Santa Bárbara de Casa,37.0,2.0,6.0,1043.0,191.754554,575.263663,0.0,0.333333
Puebla de Guzmán,148.0,1.0,3.0,3092.0,32.341527,97.024580,0.0,0.333333
Isla Cristina,2446.0,67.0,165.0,21393.0,313.186556,771.280325,5.0,0.406061
San Juan del Puerto,620.0,10.0,22.0,9411.0,106.258634,233.768994,4.0,0.454545
Almonte,1969.0,11.0,24.0,24507.0,44.885135,97.931203,2.0,0.458333
